In [2]:
import numpy as np
import WTA_CONV
import tensorflow as tf
import os
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', validation_size=10000)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [18]:
# Train config class
class train_config:
    
    epochs = 20
    batch_size = 300
    learning_rate = 0.001
    save_path = 'models/'
    save_name='model.cptk'


# Test config class     
class test_config:
    num_test_imgs = 300
    save_path = 'models/'
    save_name='model.cptk'
    


train_config = train_config()
test_config = test_config()

In [ ]:




##### Training below this line #####

tf.reset_default_graph()





In [1]:
sparcity = [0.05,0.2,0.4,0.6,0.8,0.95]

for sparse_const in sparcity:
    
    tf.reset_default_graph()
    model = WTA_CONV.WTA_CONV(mnist.train, mnist.validation, mnist.test)

    inputs = tf.placeholder(tf.float32, (None, 28, 28, 1), name='inputs')
    targets = tf.placeholder(tf.float32, (None, 28, 28, 1), name='targets')

    WTA_CONV_model = model.model(inputs,train_config,reuse = False)


    loss = model.loss(targets = targets, model = WTA_CONV_model, lifetime_sparsity = sparse_const)
    opt = tf.train.AdamOptimizer(train_config.learning_rate).minimize(loss)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for e in range(train_config.epochs):

        for ii in range(0,model.train.num_examples // train_config.batch_size, 2):
            batch = model.train.next_batch(train_config.batch_size)
            imgs = batch[0].reshape((-1, 28, 28, 1))



            batch_cost, _ = sess.run([loss, opt], feed_dict={inputs: imgs,
                                                             targets: imgs})

            print("Epoch: {}/{}...".format(e+1, train_config.epochs),
                  "Training loss: {:.4f}".format(batch_cost))
            
        saver = tf.train.Saver()
        name = 'model_' + str(e) + '_sparcity_' + str(sparse_const) + '.cptk'
        save_path = saver.save(sess, os.path.join(train_config.save_path, name))

In [ ]:





## Above this line is training ### 
tf.reset_default_graph()






In [2]:
# Initiate model and placeholders
# Array of the best models for each sparcity constant
best_models = np.array([])
best_accuracys = np.array([])

for sparse_const in sparcity:
    print("--------   ","Constant:",sparse_const,"   --------------")
    max_accuracy = 0
    best_model = -1
    best_positions = None
    
    for e in range(15):
        
        tf.reset_default_graph()
        test_config.save_name = 'model_' + str(e) + '_sparcity_' + str(sparse_const) + '.cptk'
        
        model = WTA_CONV.WTA_CONV(mnist.train, mnist.validation, mnist.test)


        inputs = tf.placeholder(tf.float32, (None, 28, 28, 1), name='inputs')
        targets = tf.placeholder(tf.float32, (None, 28, 28, 1), name='targets')

        WTA_CONV_model = model.model(inputs,train_config,False)

        loss = model.loss(targets, WTA_CONV_model, 0.2)

        # Restore model
        sess = tf.Session()
        saver = tf.train.Saver()
        saver = saver.restore(sess, os.path.join(test_config.save_path,test_config.save_name))

        # Get test data
        imgs = np.reshape(model.validation.images[:test_config.num_test_imgs],(-1,28,28,1))

        # Get embeddings
        get_embeddings = model.get_embedding(WTA_CONV_model)


        embeddings= sess.run([get_embeddings], feed_dict={inputs: imgs,
                                                          targets: imgs})



        # Embeddings returned from test data
        embeddings = np.reshape(embeddings,(test_config.num_test_imgs,4,4,8))



        predicted_labels = []
        test_config.num_test_imgs = 300
        neighbours = 10

        classifier = model.fit_KNN(embeddings, neighbours, p=2)
        print('Classifier fit. Finding nearest neighbours.')

        for query in range(test_config.num_test_imgs):
            indices = model.K_nearest_neighbours(classifier, embeddings[query])
            predicted_labels = np.append(predicted_labels,indices)

        predicted_labels = np.reshape(predicted_labels,(test_config.num_test_imgs, neighbours))

        accuracy, positions = model.model_accuracy(predicted_labels, test_config.num_test_imgs, neighbours)

        if accuracy > max_accuracy:
            best_model = test_config.save_name
            max_accuracy = accuracy
            best_positions = positions
            
    best_models = np.append(best_models,test_config.save_name)
    best_accuracys = np.append(best_accuracys,max_accuracy)
    model.plot_bar_chart(best_positions, test_config.num_test_imgs)
    print("Best model:",best_model)
        

In [3]:
print(best_models)
print(best_accuracys)

In [4]:
# Testing accuracy of feature diffusion

embed = []

for name in best_models[:4]:
    tf.reset_default_graph()
    test_config.save_name = name

    model = WTA_CONV.WTA_CONV(mnist.train, mnist.validation, mnist.test)


    inputs = tf.placeholder(tf.float32, (None, 28, 28, 1), name='inputs')
    targets = tf.placeholder(tf.float32, (None, 28, 28, 1), name='targets')

    WTA_CONV_model = model.model(inputs,train_config,False)

    loss = model.loss(targets, WTA_CONV_model, 0.2)

    # Restore model
    sess = tf.Session()
    saver = tf.train.Saver()
    saver = saver.restore(sess, os.path.join(test_config.save_path,test_config.save_name))

    # Get test data
    imgs = np.reshape(model.validation.images[:test_config.num_test_imgs],(-1,28,28,1))

    # Get embeddings
    get_embeddings = model.get_embedding(WTA_CONV_model)


    embeddings= sess.run([get_embeddings], feed_dict={inputs: imgs,
                                                      targets: imgs})



    # Embeddings returned from test data
    embeddings = np.reshape(embeddings,(test_config.num_test_imgs,4,4,8))
    embeddings = model.flatten_embeddings(embeddings)
    
    embed = np.append(embed,embeddings)
embed = np.reshape(embed,(4,test_config.num_test_imgs, 128))

In [5]:
import Collaborative_Affinity_Metric_Fusion
features_algo_1 = embed[0]
features_algo_2 = embed[1]
features_algo_3 = embed[2]
features_algo_4 = embed[3]


CAMF = Collaborative_Affinity_Metric_Fusion.collaborative_affinity_metric_fusion()

status_matrix_1, kernel_matrix_1 = CAMF.create_graphs(features_algo_1, neighbours = 20)
status_matrix_2, kernel_matrix_2 = CAMF.create_graphs(features_algo_2, neighbours = 20)
status_matrix_3, kernel_matrix_3 = CAMF.create_graphs(features_algo_3, neighbours = 20)
status_matrix_4, kernel_matrix_4 = CAMF.create_graphs(features_algo_4, neighbours = 20)


status_matrix = CAMF.merge_graph(status_matrix_1, status_matrix_2, status_matrix_3, status_matrix_4)
kernel_matrix = CAMF.merge_graph(kernel_matrix_1, kernel_matrix_2, kernel_matrix_3, kernel_matrix_4)
W_FAM = CAMF.cross_diffusion(status_matrix, kernel_matrix)
similarity_matrix = CAMF.image_similarity(W_FAM,10)

predicted_labels = []
true_labels = []

for row in similarity_matrix:
    for col in row:
        label = mnist.validation.labels[int(col)]
        predicted_labels = np.append(predicted_labels, label)

predicted_labels = np.reshape(predicted_labels,similarity_matrix.shape)

for i in range(test_config.num_test_imgs):
    true_labels = np.append(true_labels,mnist.validation.images[int(i)])

accuracy, positions = model.model_accuracy(predicted_labels, 300, 10)
model.plot_bar_chart(positions, 300)